In [ ]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np 
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import  stats
from scipy.stats import binned_statistic
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
import warnings

#methods

from sklearn.metrics import mean_squared_error
import xgboost as xgb 
from sklearn import preprocessing,svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv(r"C:\Users\ashum\Desktop\driven data  denguAI\dengue_features_train.csv")
datay = pd.read_csv(r"C:\Users\ashum\Desktop\driven data  denguAI\dengue_labels_train.csv")

In [ ]:
city = np.array(data["city"])
citty = [ 0 if i == "sj" else 1 for i in city]
data = data.drop(["city","week_start_date","year"],axis=1)
data['city'] = citty

In [ ]:
null_train = data.isnull().sum()
null_train = null_train[null_train>0]
null_train.sort_values(inplace = True)
null_train.plot.bar()

In [145]:
def preproces(data):
    #data.isnull().sum()    to check null value in dataframe
    #normal_data.dropna(inplace=True)
    normal_data = (data - data.mean())/data.std()
    datax=normal_data.fillna(normal_data.mean())
    return datax
datay = datay["total_cases"]
#total_dat = datax
#total_dat["total_cases"] = datay
datax = preproces(data)

In [ ]:
total_dat = datax.copy()
total_dat["total_cases"] = datay
print(total_dat.columns)
print(datax.columns)

In [ ]:
#Using Pearson Correlation
sns.heatmap(total_dat.corr(),vmax=0.7,square=True,linewidth=0.6,cmap="BuPu")
#sns.heatmap(datax.corr(),annot=True,fmt='d',cmap="BuPu"(thischangescolor))  to get heat map with values

In [ ]:
#Correlation with output variable
cor_target=abs(total_dat.corr()["total_cases"])


In [ ]:
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.1]
relevant_features

In [ ]:
print(total_dat[["weekofyear","city","station_min_temp_c"]].corr())


In [ ]:
print(total_dat[["reanalysis_air_temp_k","ndvi_sw"]].corr())


In [ ]:
'''
c
reanalysis_air_temp_k
city
weekofyear
station_min_temp_c   #notclear
ndvi_sw
ndvi_se
'''

In [ ]:
col_heatmap = ['reanalysis_air_temp_k','city','weekofyear','station_min_temp_c','ndvi_sw','ndvi_se']


In [ ]:
'''
Wrapper Method :A wrapper method needs one machine learning algorithm and uses its performance as evaluation criteria. This means, you feed the features to the selected Machine Learning algorithm and based on the model performance you add/remove the features. This is an iterative and computationally expensive process but it is more accurate than the filter method.

There are different wrapper methods such as Backward Elimination, Forward Selection, Bidirectional Elimination and RFE.
'''

In [ ]:
'''
Backward Elimination

The performance metric used here to evaluate feature performance is pvalue. If the pvalue is above 0.05 then we remove the feature, else we keep it.
'''

#Adding constant column of ones, mandatory for sm.OLS model
x1 = sm.add_constant(datax)

#Fitting sm.OLS model
model_backward_elimination = sm.OLS(datay,x1).fit()
model_backward_elimination.pvalues

In [ ]:
#Backward Elimination
cols = list(datax.columns)
pmax = 1
while(len(cols)>0):
    p=[]
    x1 = datax[cols]
    x1 = sm.add_constant(x1)
    model_backward_elimination = sm.OLS(datay,x1).fit()
    p = pd.Series(model_backward_elimination.pvalues.values[1:],index=cols)
    pmax=max(p)
    feature_pmax = p.idxmax()
    if (pmax > 0.5):
        cols.remove(feature_pmax)
    else:
        break
col_backward_elimination = cols
print(col_backward_elimination)
print(len(col_backward_elimination))
print(len(data.columns))

In [ ]:
'''
RFE (Recursive Feature Elimination)

The Recursive Feature Elimination (RFE) method works by recursively removing attributes and building a model on those attributes that remain. It uses accuracy metric to rank the feature according to their importance. The RFE method takes the model to be used and the number of required features as input. It then gives the ranking of all the variables, 1 being most important. It also gives its support, True being relevant feature and False being irrelevant feature.
'''

In [ ]:
model = LinearRegression()

#Initializing RFE model
rfe = RFE(model,7)

#Transforming data using RFE
x_rfe = rfe.fit_transform(datax,datay)


#Fitting the data to model
model.fit(x_rfe,datay)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
#no of features
nof_list = np.arange(1,22)
high_score=0
#Variable to store the optimum features
nof =0
score_list=[]
for n in range(len(nof_list)):
    x_train,x_test,y_train,y_test = train_test_split(datax,datay,test_size = 0.3,random_state = 1)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    x_train_rfe = rfe.fit_transform(x_train,y_train)
    x_test_rfe = rfe.transform(x_test)
    model.fit(x_train_rfe,y_train)
    score = model.score(x_test_rfe,y_test)
    score_list.append(score)
    if (score > high_score):
        high_score = score
        nof = nof_list[n]

print("optimum no of features: %d"%nof)
print("score with %d features: %f"%(nof,high_score))

In [ ]:
cols = list(datax.columns)
model = LinearRegression()

#Initializing RFE model
rfe = RFE(model,9)

#Transforming data using RFE
x_rfe = rfe.fit_transform(datax,datay)

#Fitting the data to model
model.fit(x_rfe,datay)
temp = pd.Series(rfe.support_,index = cols)
col_rfe = temp[temp==True].index
print(col_rfe)
print(len(col_rfe))

In [ ]:
'''
Embedded Method

Embedded methods are iterative in a sense that takes care of each iteration of the model training process and carefully extract those features which contribute the most to the training for a particular iteration. Regularization methods are the most commonly used embedded methods which penalize a feature given a coefficient threshold.
Here we will do feature selection using Lasso regularization. If the feature is irrelevant, lasso penalizes it’s coefficient and make it 0. Hence the features with coefficient = 0 are removed and the rest are taken.
'''


In [ ]:
reg = LassoCV()
reg.fit(datax,datay)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(datax,datay))
coef = pd.Series(reg.coef_, index = datax.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

In [ ]:
'''

1. Filter method(like heatmap filteration) is less accurate. It is great while doing EDA, it can      also be used for checking multi co-linearity in data.
2. Wrapper and Embedded methods give more accurate results but as they      are computationally expensive, these method are suited when you have      lesser features (~20).
'''

In [ ]:
col_heatmap
col_backward_elimination
col_rfe

In [ ]:
train_meta = total_dat[col_heatmap].copy()
train_meta.head(5)
train_meta['total_cases'] = datay

In [ ]:
sns.pairplot(train_meta)

In [ ]:
'''
outlier removal

'''

In [ ]:
train_meta.columns

In [ ]:
a = set(train_meta[(train_meta['reanalysis_air_temp_k'] > 0) & (train_meta['total_cases'] > 300)].index.tolist())
b = set(train_meta[(train_meta['weekofyear'] > 0.5) & (train_meta['total_cases'] > 300)].index.tolist())
c = set(train_meta[(train_meta['station_min_temp_c'] > 0) & (train_meta['total_cases'] > 300)].index.tolist())
d = set(train_meta[(train_meta['ndvi_sw'] > -1 ) & (train_meta['ndvi_sw'] < 1 ) & (train_meta['total_cases'] > 300)].index.tolist())
e = set(train_meta[(train_meta['ndvi_se'] > -1 ) & (train_meta['ndvi_se'] < 1 ) & (train_meta['total_cases'] > 300)].index.tolist())


In [ ]:
unnecessary = a.union(b).union(c).union(d).union(e)
unnecessary = list(unnecessary)
train_clean = train_meta.drop(unnecessary)

In [ ]:
y = train_clean['total_cases']
x = train_clean.drop(['total_cases'],axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=1)
reg = LinearRegression()
reg.fit(x_train,y_train)
print(reg.score(x_test,y_test))

In [ ]:
train_rfe = total_dat[col_rfe]
train_rfe['total_cases'] = datay

In [ ]:
train_rfe.columns

In [ ]:
sns.pairplot(train_rfe)

In [ ]:
a = set(train_rfe[(train_rfe['reanalysis_air_temp_k'] > 0) & (train_rfe['total_cases'] > 200)].index.tolist())
b = set(train_rfe[(train_rfe['weekofyear'] > 0.5) & (train_rfe['total_cases'] > 200)].index.tolist())
c = set(train_rfe[(train_rfe['station_max_temp_c'] > 0) & (train_rfe['total_cases'] > 200)].index.tolist())
d = set(train_rfe[(train_rfe['reanalysis_avg_temp_k'] > -1 ) & (train_rfe['reanalysis_avg_temp_k'] < 1 ) & (train_rfe['total_cases'] > 200)].index.tolist())
e = set(train_rfe[(train_rfe['reanalysis_dew_point_temp_k'] > 0 ) & (train_rfe['total_cases'] > 200)].index.tolist())
f = set(train_rfe[(train_rfe['reanalysis_relative_humidity_percent'] > -1 ) & (train_rfe['reanalysis_relative_humidity_percent'] < 1 ) & (train_rfe['total_cases'] > 200)].index.tolist())
g = set(train_rfe[(train_rfe['reanalysis_specific_humidity_g_per_kg'] > -1 ) & (train_rfe['reanalysis_specific_humidity_g_per_kg'] < 1 ) & (train_rfe['total_cases'] > 200)].index.tolist())
h = set(train_rfe[(train_rfe['reanalysis_tdtr_k'] < 0 ) & (train_rfe['total_cases'] > 200)].index.tolist())


In [ ]:
unnecessary_rfe = a.union(b).union(c).union(d).union(e).union(f).union(g).union(h)
unnecessary_rfe = list(unnecessary_rfe)
train_clean_rfe = train_meta.drop(unnecessary)

In [ ]:
y = train_clean_rfe['total_cases']
x = train_clean_rfe.drop(['total_cases'],axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)


In [ ]:
yy_train=y
xx_train=x

In [ ]:
xg_reg = xgb.XGBRegressor(objective='reg:linear',colsample_bytree=0.3,learning_rate = 0.1,max_depth=5, alpha=10, n_estimators=1000)

In [ ]:
xg_reg.fit(x_train,y_train)
preds = xg_reg.predict(x_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test,preds))
print("RMSE: %f"%(rmse))

In [ ]:
xg_reg.fit(xx_train,yy_train)
preds = xg_reg.predict(xx_train)
rmse = np.sqrt(mean_squared_error(yy_train,preds))
print("RMSE: %f"%(rmse))

In [ ]:
test_data = pd.read_csv(r"C:\Users\ashum\Desktop\driven data  denguAI\dengue_features_test.csv")